<a href="https://colab.research.google.com/github/amyfthau/CFGSum23D1G4/blob/data_collection/ONS_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ONS API Data Retrieval Notebook


This Jupyter Notebook demonstrates how to retrieve and analyse data from the Office for National Statistics (ONS) API using Python.

While the ONS is currently in the process of adding some information into their API and are from having added all of its datasets onto the API, there are over 300 datasets on that page.

The supporting API documentation is available on:

https://digitalblog.ons.gov.uk/2021/02/15/how-to-access-data-from-the-ons-beta-api/

Access to the API is free with no registration required. It returns data in a JSON format with some rate limits.

This notebook makes retrieval more simple and intuitive.

## Some Advantages of using the API
1. Provides access to real-time and dynamically updated data. This is particularly useful for our datasets which are frequently updated. Therefore, by using the API, we can always get the most up-to-date figures by running the script rather than checking on the website
2. The API also allows for automated data retrieval: APIs allow you to automate the process of data retrieval. This is especially advantageous for ongoing monitoring and analysis tasks.

## Terminology
Descriptions for some terms used within this API.

1. Dataset:A grouping of data (editions) with shared dimensions, for example Sex, Age and Geography, and all published history of this group of data. The options in these dimensions can change over time leading to separate editions. For example: Population Estimates for UK, England and Wales, Scotland and Northern Ireland

2. Edition: A subset of the dataset representing a specific time period. For some datasets this edition can contain all time periods (all historical data). The latest version of this is displayed by default. For example: Population Estimates for UK, England and Wales, Scotland and Northern Ireland Mid-2016

3. Version: A specific instance of the edition at a point in time. New versions can be published as a result of corrections, revisions or new data becoming available.

## Getting Started
Follow these steps to get started with the notebook:

Clone or download this repository to your local machine.

Ensure you have the necessary Python packages installed. You can install them using the following command:

`pip install pandas requests`

Open the Jupyter Notebook `ONS_API_Data_Retrieval.ipynb` in your Jupyter environment.

## Functions for Dataset Exploration

Before extracting data from the ONS API, this notebook provides functions to explore available datasets, parameters, options, versions, and editions:

1. Run the `retrieve_all_datasets` function to find a list of datasets and their IDs and their description.
2. Declare the `dataset_id` variable with your desired dataset ID.
3. Utilise the following functions to retrieve relevant editions, versions, parameters, and options for your chosen `dataset_id` (there are some prespecified dataset_ids). These functions will also display information about the available parameters and their corresponding options, tailored to your specific `dataset_id`. This exploration is crucial for constructing precise data queries.

The exploration functions include:

### Retrieving Dataset Information
To retrieve a list of available datasets along with their descriptions, use the `retrieve_all_datasets` function.

### Finding Available Editions
You can find available editions for a specific dataset using the `get_ONS_API_editions`

### Finding the Latest Version
To get the latest version of a dataset, use the `get_ONS_API_latest_ver function`

### Exploring Parameters and Options
To explore available parameters and their corresponding options for a specific dataset, version, and edition, use the `get_ONS_API_param_info function`

## Data Retrieval

After retrieving and storing editions:

1. Set the base URL for the ONS API
2. Define query parameters to retrieve data based on the functions above eg. for life expectancy:

```
query_params = {
    "agegroups": "30-34",
    "time": "*",
    "sex": "female",
    "geography": "E06000001"
}
```
3. Call the `retrieve_ONS_API_data` function to retrieve the data and store it in a Pandas.

You can only use one wildcard *

```
ONS_API_life_exp = retrieve_ONS_API_data(dataset_id, ONS_base_url, query_params, latest_version, available_edition)
```

### Enhancements
Feel free to enhance this notebook further by incorporating additional explanations, examples, or custom analysis based on your specific use case.


**Mount Google Drive and Change Directory**

In [1]:
# Import relevant libraries to mount drive
from google.colab import drive
import os

# Mount google drive
drive.mount('/content/drive')

# Define the path to your specific folder
specific_folder_path = '/content/drive/MyDrive/CFGdegree_Sum23_D1_G4'

# Navigate to the specific folder
os.chdir(specific_folder_path)

# change directory to shared group folder
%cd /content/drive/MyDrive/CFGdegree_Sum23_D1_G4




Mounted at /content/drive
/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import Relevant Libraries**

In [ ]:
# Import pandas
import pandas as pd
# Import requests
import requests
# # Import unittest
# import unittest
# Import date class from datetime module
from datetime import date


**Functions**:

Comprehensive script for interacting with the ONS API to retrieve dataset information, editions, versions, parameters, and options.

The script will allow us to:

* Check API access and retrieve dataset information.
* Get available editions for a dataset.
* Retrieve the latest version of a dataset.
* Retrieve parameter information and options for a specific version of a dataset.
* and show the list of datasets available: there is a function to download a csv file (see Drive) of the dataset IDs, and then the rest of the functions allow us to automatically go to the latest version's endpoint and provide a set of parameters and options for this.

In [ ]:
# Define the base URL
ONS_base_url = "https://api.beta.ons.gov.uk/v1"

# Function to retrieve all datasets and their descriptions
def retrieve_all_datasets(ONS_base_url):
    """
    Retrieve all available datasets from the ONS API along with their descriptions.

    Args:
        ONS_base_url (str): The API base URL.

    Returns:
        pd.DataFrame: A DataFrame containing dataset information (id, description).
    """
    # Call the function to check API access
    request_success_ONS_api(ONS_base_url)

    datasets = []

    # Initialize pagination parameters
    limit = 1000
    offset = 0

    while True:
        # Set endpoint with pagination parameters
        ONS_endpoint = f"/datasets?limit={limit}&offset={offset}"

        # Retrieve list of datasets / send GET request to the specified API endpoint
        response = requests.get(ONS_base_url + ONS_endpoint)

        # Check if the request was successful (HTTP status code = 200 if successful)
        if response.status_code == 200:
            # Change response to JSON format
            data = response.json()
            # Extract the "items" field from the JSON
            items = data.get("items", [])
            datasets.extend(items)

            # If we have reached the end of the list, break the loop
            if len(items) < limit:
                break

            # Increment offset for the next page
            offset += limit
        else:
            print('Request for dataset information failed with status code:', response.status_code)
            break

    # Create a list of dictionaries with id and description
    dataset_info = [{"id": item["id"], "description": item["description"]} for item in datasets]

    # Convert the list of dictionaries into a Pandas DataFrame
    datasets_df = pd.DataFrame(dataset_info)
    return datasets_df


# Create function to check API access
def request_success_ONS_api(ONS_base_url):
    """
    Check if the API request is successful and return the response data.

    Args:
        ONS_base_url (str): The base URL of the ONS API.

    Returns:
        dict: The response data as a dictionary, or an empty dictionary if the request failed.
    """
    # Initialize the response data dictionary
    ONS_data_response = {}

    # Request data from the URL
    response = requests.get(ONS_base_url)

    # Check if the request was successful (HTTP status code = 200 if successful)
    if response.status_code == 200:
        # Convert response to JSON (dictionary)
        ONS_data_response = response.json()
        print(f"Request success with status code: {response.status_code}")
    else:
        print('Request failed with status code:', response.status_code)

    return ONS_data_response

# Create function to access first <limit> number of datasets available on ONS
def retrieve_ONS_API_datasets_desc(ONS_base_url, limit):
    """
    Retrieve dataset information from the ONS API and create a Pandas DataFrame.

    Args:
        ONS_base_url (str): The API base URL.
        limit (int): The number of datset id and descriptions to view. Maximum is 1000

    Returns:
        pd.DataFrame: A DataFrame containing dataset information (id, description).
    """
    # Call the function to check API access
    request_success_ONS_api(ONS_base_url)

    # Set endpoint to list of datasets available for API
    ONS_endpoint = f"/datasets?limit={limit}"

    # Retrieve list of datasets / send GET request to the specified API endpoint
    response = requests.get(ONS_base_url + ONS_endpoint)

    # Check if the request was successful (HTTP status code = 200 if successful)
    if response.status_code == 200:
        # Change response to JSON format
        data = response.json()

        # Extract the "items" field from the JSON
        # The square brackets [] are used to provide a default value in case the key "items"
        items = data.get("items", [])

        # Create list of dictionaries with "id" "description"
        # (list of dictionaries is easily converted into a Pandas DataFrame)
        dataset_info = [{"id": item["id"], "description": item["description"]} for item in items]

        # Change list of dictionaries into a Pandas DataFrame
        ONS_API_datasets_desc_df = pd.DataFrame(dataset_info)
        return ONS_API_datasets_desc_df

    else:
        print('Request for dataset information failed with status code:', response.status_code)
        return None

# Create function to convert Pandas DataFrame into CSV files
def save_df_to_csv(dataframe, file_name):
    """
    Convert and save a Pandas DataFrame to a CSV file.

    Args:
        dataframe (pd.DataFrame): DataFrame to be saved.
        file_name (str): Name of the CSV file

    Returns:
        csv_file (csv): "ONS_API_available_datasets"
    """
    df = dataframe.to_csv(file_name, index=False)
    print(f"DataFrame saved as {file_name}.csv")
    return None

# Create function to retrieve editions for an IDs
def get_ONS_API_editions(dataset_id, ONS_base_url):
    """
    Retrieve dataset editions from the ONS API and create a Pandas DataFrame.

    Args:
        dataset_id (str): the ID for the dataset to extract the available editions
        ONS_base_url (str): The API base URL.

    Returns:
        edition (str): provides the latest edition number eg. `37`

    Note: to see all editions, comment out `return edition` and
    uncomment `return editions_df` and adjust the variable calling the edition

    """
    # Call the function to check API access
    request_success_ONS_api(ONS_base_url)

    # Set endpoint to list of datasets available for API
    ONS_endpoint = f"/datasets/{dataset_id}/editions"

    # Retrieve list of datasets / send GET request to the specified API endpoint
    response = requests.get(ONS_base_url + ONS_endpoint)

    # Check if the request was successful (HTTP status code = 200 if successful)
    if response.status_code == 200:
        # Change response to JSON format
        data = response.json()
        # Extract the "items" field from the JSON
        # The square brackets [] are used to provide a default value in case the key "items"
        items = data["items"]
        # Extract edition
        edition = [item["edition"] for item in items]
        # Put into Pandas DataFrame
        editions_df = pd.DataFrame({"Edition": [edition]})
        # Retrieve latest edition by getting last value in dataframe
        edition = editions_df.iloc[-1][0]
        # Convert to string
        edition = str(edition[0])
        # return editions_df
        return edition
    else:
        print('Request for dataset information failed with status code:', response.status_code)
        return None

# Create function to retrieve latest version of database
def get_ONS_API_latest_ver(dataset_id, ONS_base_url, edition):
    """
    Retrieve dataset latest version of dataset from the ONS API and create a Pandas DataFrame.

    Args:
        dataset_id (str): the ID for the dataset to extract the latest version
        ONS_base_url (str): The API base URL
        edition (str): the version to use.

    Returns:
        latest version (int): the latest version of the dataset eg `time-series`
    """
    # Call the function to check API access
    request_success_ONS_api(ONS_base_url)

    # Set endpoint to list of datasets available for API
    ONS_endpoint = f"/datasets/{dataset_id}/editions/{edition}"

    # Retrieve list of datasets / send GET request to the specified API endpoint
    response = requests.get(ONS_base_url + ONS_endpoint)

    # Check if the request was successful (HTTP status code = 200 if successful)
    if response.status_code == 200:
        # Change response to JSON format
        data = response.json()
        # Extract latest version
        latest_version = data["links"]['latest_version']['id']
        # Put into Pandas DataFrame
        latest_version_df = pd.DataFrame({"Latest Version": [latest_version]})
        # Retrieve the latest version
        latest_version = str(latest_version_df.iloc[0][0])
        return latest_version
    else:
        print('Request for dataset information failed with status code:', response.status_code)
        return None

def get_ONS_API_options(dataset_id, latest_version, parameter, edition):
    """
    Retrieve options for a specific parameter of a dataset from the ONS API.
    This is used within the `get_ONS_API_param_info` function for each parameter

    Args:
        dataset_id (str): the ID for the dataset
        latest_version (str): the latest version of the dataset
        parameter (str): the name of the parameter for which to retrieve options
        edition (str): the edition of the dataset.

    Returns:
        options: A list of options for the specified parameter

        , or None if the request fails.
    """

    # Call the function to check API access
    request_success_ONS_api(ONS_base_url)

    # Set endpoint to retrieve options for the specified parameter
    ONS_endpoint = f"/datasets/{dataset_id}/editions/{edition}/versions/{latest_version}/dimensions/{parameter}/options"

    # Retrieve options data / send GET request to the specified API endpoint
    response = requests.get(ONS_base_url + ONS_endpoint)

    # Check if the request was successful (HTTP status code = 200 if successful)
    if response.status_code == 200:
        # Change response to JSON format
        data = response.json()
        # Extract the "items" field from the JSON
        options = [{"option": item["option"], "label": item["label"]} for item in data["items"]]
        return options
    else:
        print('Request for options information failed with status code:', response.status_code)
        return None


# Create function to retrieve parameters/dimensions and their corresponding options for an ID
def get_ONS_API_param_info(dataset_id, ONS_base_url, latest_version):
    """
    Retrieve dimension information for a specific version of a dataset from the ONS API and create a Pandas DataFrame.

    Args:
        dataset_id (str): the ID for the dataset
        ONS_base_url (str): The API base URL.
        latest_version (str): the version to use.

    Returns:
        ONS_endpoint (str): endpoint of the relevant dataset_id based on the latest versions and Edition
                            this is useful if the function to retrieve observations does not work
        options_df (pd.DataFrame): A DataFrame containing dimension information.

        , or None if the request fails.

    """
    # Call the function to check API access
    request_success_ONS_api(ONS_base_url)

    # Set endpoint to retrieve dimensions for the specified version
    ONS_endpoint = f"/datasets/{dataset_id}/editions/{edition}/versions/{latest_version}/dimensions"

    # Retrieve dimension data / send GET request to the specified API endpoint
    response = requests.get(ONS_base_url + ONS_endpoint)

    # Check if the request was successful (HTTP status code = 200 if successful)
    if response.status_code == 200:
        # Change response to JSON format
        data = response.json()
        # Extract the "items" field from the JSON
        items = data["items"]
        # Loop through all items to get params
        params = [item["name"] for item in items]
        # Create an empty list to store options for each parameter
        options_list = []
        # Loop through each parameter to get options
        for param in params:
            options = get_ONS_API_options(dataset_id, latest_version, param, edition)
            options_list.append(options)
        # Put into Pandas DataFrame
        options_df = pd.DataFrame({"Parameters": params, "Options": options_list})
        return ONS_endpoint, options_df
    else:
        print('Request for dimension information failed with status code:', response.status_code)
        return None

def get_dataset_description(ONS_base_url, dataset_id):
    """
    Retrieve the description of a dataset from the ONS API.

    Args:
        ONS_base_url (str): The API base URL.
        dataset_id (str): The ID of the dataset.

    Returns:
        str: The description of the dataset, or None if the request fails.
    """
    # Call the function to check API access
    request_success_ONS_api(ONS_base_url)

    # Set endpoint to retrieve dataset information
    ONS_endpoint = f"/datasets/{dataset_id}"

    # Retrieve dataset information / send GET request to the specified API endpoint
    response = requests.get(ONS_base_url + ONS_endpoint)

    # Check if the request was successful (HTTP status code = 200 if successful)
    if response.status_code == 200:
        # Change response to JSON format
        data = response.json()
        # Extract and return the description
        description = data.get("description")
        return description
    else:
        print('Request for dataset description failed with status code:', response.status_code)
        return None


# Function to retrieve data from ONS API
def retrieve_ONS_API_data(dataset_id, ONS_base_url, query_params, version, edition):
    """
    Retrieve data from the ONS API for a specific dataset, version, and edition, based on given query parameters.

    Args:
        dataset_id (str): The ID of the dataset to retrieve data from.
        ONS_base_url (str): The base URL of the ONS API.
        query_params (dict): Query parameters to filter the data request filled with strings
        version (str): The version of the dataset.
        edition (str): The edition of the dataset.

    Returns:
        pd.DataFrame: A DataFrame containing retrieved data (Date, Data).
    """
    # Call the function to check API access
    request_success_ONS_api(ONS_base_url)

    # Set params for API request
    query_params = query_params

    # Construct the API request URL
    ONS_endpoint = f"/datasets/{dataset_id}/editions/{edition}/versions/{version}/observations"

    # Send GET request to the API endpoint
    response = requests.get(ONS_base_url + ONS_endpoint, params=query_params)

    # Create lists to store data
    dates = []
    data_values = []

    # Process the response
    if response.status_code == 200:
        ONS_data_response = response.json()
        observations = ONS_data_response["observations"]

        # for observation in observations:
        #     date = observation['dimensions']['Time']['label']
        #     data = observation['observation']
        #     dates.append(date)
        #     data_values.append(data)

        for observation in observations:
            # Check if 'Time' key exists, otherwise try 'time'
            if 'Time' in observation['dimensions']:
                date = observation['dimensions']['Time']['label']
            elif 'time' in observation['dimensions']:
                date = observation['dimensions']['time']['label']
            else:
                date = None
            data = observation['observation']
            dates.append(date)
            data_values.append(data)


        # Create a DataFrame
        ONS_API_df = pd.DataFrame({'Date': dates, 'Data': data_values})
        # Check if DataFrame is retrieved
        if ONS_API_df is not None:
          print("Retrieved data:")
          print(ONS_API_df)
        # Return df
        return ONS_API_df

    else:
        print('Request failed with status code:', response.status_code)
        return None

**Call functions to download into drive the available datasets**

In [ ]:
# Set parameters
# Set limit of datasets loaded
limit = 1000
# Set ONS base URL
ONS_base_url = "https://api.beta.ons.gov.uk/v1"
# Returns the current local date
today = str(date.today())

# Call function to retrieve datasets
all_datasets_df = retrieve_all_datasets(ONS_base_url)

# Call function to convert to CSV file
save_df_to_csv(all_datasets_df, f"ONS_API_available_datasets_{today}.csv")

Request success with status code: 200
DataFrame saved as ONS_API_available_datasets_2023-08-20.csv.csv


**Call functions to show parameters, available options, versions, editions and data description etc for retrieval of data**

Modify the dataset_id to the appropriate ID to check for parameters

In [ ]:
# Set ID
# dataset_id = "cpih01"
# dataset_id = "regional-gdp-by-quarter"
# dataset_id = "house-prices-local-authority"
# dataset_id = "gdp-to-four-decimal-places"
# dataset_id = "weekly-deaths-region"
# dataset_id = "retail-sales-index"
# dataset_id = "online-job-advert-estimates"
dataset_id = "life-expectancy-by-local-authority"
# dataset_id = "house-prices-local-authority" # starts from 2015
# dataset_id = "index-private-housing-rental-prices" # only starts from dec-21
# dataset_id = "labour-market" # unemployment indicator


# A) Retrieve last editions
edition = get_ONS_API_editions(dataset_id, ONS_base_url)

# B) Retrieve latest version of dataset
latest_version = get_ONS_API_latest_ver(dataset_id, ONS_base_url, edition)
version = latest_version

# C) Retrieve information around parameters
ons_endpoint_result, options_df = get_ONS_API_param_info(dataset_id, ONS_base_url, latest_version)
# print(options_df)

# Print parameters and options
for index, row in options_df.iterrows():
    parameter = row["Parameters"]
    options = row["Options"]

    # Print the parameter name
    print(f"\nParameter: {parameter}")

    # Loop through options for each parameter
    for option in options:
        option_name = option['option']
        option_label = option['label']
        # Indent and print each option and its label
        print(f"    ⟡ Option: '{option_name}', ⟡ Label: '{option_label}'")

    # Print a new line to separate parameters
    print("\n")

# D) Retrive dataset description
description = get_dataset_description(ONS_base_url, dataset_id)

# E) Print all results
print(f"**********\nThis is information for dataset '{dataset_id}' ")
print(f"⟡ With the following description: {description}")
print(f"⟡ The latest version of this dataset is '{version}'")
print(f"⟡ The edition is: '{edition}'")
print(f"⟡ The final endpoint is:'{ons_endpoint_result}'")
print("**********")

Request success with status code: 200
Request success with status code: 200
Request success with status code: 200
Request success with status code: 200
Request success with status code: 200
Request success with status code: 200
Request success with status code: 200

Parameter: agegroups
    ⟡ Option: '0-1', ⟡ Label: '00-01'
    ⟡ Option: '1-4', ⟡ Label: '01-04'
    ⟡ Option: '10-14', ⟡ Label: '10-14'
    ⟡ Option: '15-19', ⟡ Label: '15-19'
    ⟡ Option: '20-24', ⟡ Label: '20-24'
    ⟡ Option: '25-29', ⟡ Label: '25-29'
    ⟡ Option: '30-34', ⟡ Label: '30-34'
    ⟡ Option: '35-39', ⟡ Label: '35-39'
    ⟡ Option: '40-44', ⟡ Label: '40-44'
    ⟡ Option: '45-49', ⟡ Label: '45-49'
    ⟡ Option: '5-9', ⟡ Label: '05-09'
    ⟡ Option: '50-54', ⟡ Label: '50-54'
    ⟡ Option: '55-59', ⟡ Label: '55-59'
    ⟡ Option: '60-64', ⟡ Label: '60-64'
    ⟡ Option: '65-69', ⟡ Label: '65-69'
    ⟡ Option: '70-74', ⟡ Label: '70-74'
    ⟡ Option: '75-79', ⟡ Label: '75-79'
    ⟡ Option: '80-84', ⟡ Label: '80-84

In [ ]:
print(dataset_id)
print(edition)
print(version)

life-expectancy-by-local-authority
time-series
1


**`Retreive data for CPI** - run when dataset set as cpih01


In [ ]:
# Define the base URL
ONS_base_url = "https://api.beta.ons.gov.uk/v1"

# Assuming we are only going to get time series (based on the nature of the analysis)
# We will set time to a wildcard to get all the data available over time
query_params = {
    "geography": "K02000001",
    "aggregate": "CP00",
    "time": "*"}


# Call function to print pandas df
ONS_API_CPI_df = retrieve_ONS_API_data(dataset_id, ONS_base_url, query_params, version, edition)
# Convert Pandas DataFrame into CSV
save_df_to_csv(ONS_API_CPI_df, "CPIH.csv")


Request success with status code: 200
Request failed with status code: 400


AttributeError: ignored

**Retreive data for GDP** - only run when dataset_id = "regional-gdp-by-quarter"

In [ ]:

# Define the base URL
ONS_base_url = "https://api.beta.ons.gov.uk/v1"

# Assuming we are only going to get time series (based on the nature of the analysis)
# We will set time to a wildcard to get all the data available over time
# england only
query_params = {
    "geography": "UK0",
    "growthrate": "grq",
    "prices" : "cvm",
    "time": "*" ,
    "unofficialstandardindustrialclassification": "A--T"}

ONS_API_GDP_df = retrieve_ONS_API_data(dataset_id, ONS_base_url, query_params, version, edition)


Request success with status code: 200
Request failed with status code: 400


**Retreive data for GDP to 4 dp** - only run when dataset_id = "gdp-to-four-decimal-places"

In [ ]:

# Define the base URL
ONS_base_url = "https://api.beta.ons.gov.uk/v1"

# Assuming we are only going to get time series (based on the nature of the analysis)
# We will set time to a wildcard to get all the data available over time
query_params = {
    "geography": "K02000001",
    "unofficialstandardindustrialclassification": "A--T",
    "time": "*"}


# Call function to print pandas df
ONS_API_GDP_4_df = retrieve_ONS_API_data(dataset_id, ONS_base_url, query_params, version, edition)


Request success with status code: 200
Request failed with status code: 400


**House Prices** dataset_id = "house-prices-local-authority"


In [ ]:
# Define the base URL
ONS_base_url = "https://api.beta.ons.gov.uk/v1"

# Assuming we are only going to get time series (based on the nature of the analysis)
# We will set time to a wildcard to get all the data available over time
query_params = {
    "buildstatus": "all",
    "geography": "E06000001",
    "housesalesandprices": "median",
    "month": "dec",
    "propertytype": "all",
    "time": "*"
    }


# Call function to print pandas df
ONS_API_HPI = retrieve_ONS_API_data(dataset_id, ONS_base_url, query_params, version, edition)

Request success with status code: 200
Request failed with status code: 400


labour-market

In [ ]:
# Define the base URL
ONS_base_url = "https://api.beta.ons.gov.uk/v1"

# Assuming we are only going to get time series (based on the nature of the analysis)
# We will set time to a wildcard to get all the data available over time
query_params = {"sex" : "all-adults",
                "time" : "*",
                "unitofmeasure": "levels",
                "seasonaladjustment": "seasonal-adjustment",
                "geography" : "K02000001",
                "economicactivity" : 'unemployed',
                "agegroups" : '16+'
    }


# Call function to print pandas df
ONS_API_labour_market = retrieve_ONS_API_data(dataset_id, ONS_base_url, query_params, version, edition)

Request success with status code: 200
Request failed with status code: 400


Life expectancy

In [ ]:
# Define the base URL
ONS_base_url = "https://api.beta.ons.gov.uk/v1"

# Assuming we are only going to get time series (based on the nature of the analysis)
# We will set time to a wildcard to get all the data available over time
query_params = {"agegroups" : "30-34",
                "time" : "*",
                "sex": "female",
                "geography" : "E06000001"
    }


# Call function to print pandas df
ONS_API_life_exp = retrieve_ONS_API_data(dataset_id, ONS_base_url, query_params, version, edition)

Request success with status code: 200
Retrieved data:
       Date   Data
0   2006-08  50.18
1   2009-11  51.57
2   2001-03   49.2
3   2004-06  49.57
4   2010-12  51.89
5   2012-14  52.15
6   2017-19  51.66
7   2002-04  49.24
8   2008-10  51.44
9   2007-09  50.66
10  2003-05  49.49
11  2005-07  49.61
12  2011-13  51.89
13  2013-15  51.85
14  2016-18  51.76
15  2015-17  51.75
16  2014-16  51.82
